In [29]:
import autodiff.admath.admath as math
from autodiff.interface.interface import AutoDiff as AD

In [3]:
def square_fn(x):
    return x ** 2

ad_square = AD(square_fn)
ad_square.get_der(3)

6

Correct!

In [4]:
ad_square.get_der([1,2])

[2, 4]

Correct!

In [5]:
def sin_fn(x):
    return math.sin(x)

ad_sin = AD(sin_fn)
ad_sin.get_der(0)

1.0

Correct!

In [6]:
def mul_fn(x, y):
    return x**2 * y**2

mul_fn = AD(mul_fn)
mul_fn.get_der([2, 2])

[16, 16]

Correct!

In [8]:
def mul_fn(x, y):
    return [x**2 * y**2, x + y]

mul_fn = AD(mul_fn)
mul_fn.get_der([2, 2])

AttributeError: 'list' object has no attribute 'der'

In [32]:
!pytest --doctest-modules --cov-report term-missing autodiff

============================= test session starts ==============================
platform darwin -- Python 3.6.6, pytest-3.8.0, py-1.6.0, pluggy-0.7.1
rootdir: /Users/pwong/.Trash/project-working 11.07.54 PM, inifile: setup.cfg
plugins: remotedata-0.3.0, openfiles-0.3.0, doctestplus-0.1.3, cov-2.6.0, arraydiff-0.2
collected 0 items / 4 errors                                                   
Coverage.py warning: Module roots was never imported. (module-not-imported)
Coverage.py warning: No data was collected. (no-data-collected)


==================================== ERRORS ====================================
_________________ ERROR collecting autodiff/test/test_dual.py __________________
autodiff/test/test_dual.py:2: in <module>
    import dual
E   ModuleNotFoundError: No module named 'dual'
_________________ ERROR collecting autodiff/test/test_dual.py __________________
ImportError while importing test module '/Users/pwong/.Trash/project-working 11.07.54 PM/autodiff/test/test_dual.

In [2]:
# from AutoDiff import AutoDiff as AD
# import admath


def square_fn(x):
    return x ** 2


def sin_fn(x):
    return admath.sin(5*x)+2*x**2

ad_square = AD(square_fn)
print(ad_square.get_der(3))

print(ad_square.get_der([1,2]))

ad_sin = AD(sin_fn)
print(ad_sin.get_der(2))
# Evaluates derivative of sin(5x)+2x^2 when x = 2


NameError: name 'AD' is not defined

In [8]:
import numpy as np

class Dual:
    
    def __init__(self, x, der=1):
        self.val = x
        self.der = der
        
    ## UNARY OPERATIONS
        
        
    def __neg__(self):
        return Dual(-self.val, -self.der)
    
    def __pos__(self):
        return Dual(+self.val, +self.der)
        
        
    ## PLUS OPERATIONS
    
    def __add__(self, other):
        try:
            return Dual(self.val + other.val, self.der + other.der)
        except AttributeError:
            return Dual(self.val + other, self.der)
    
    def __radd__(self, other):
        return Dual(other + self.val, self.der)
    
    
    ## MINUS OPERATIONS
    
    def __sub__(self, other):
        try:
            return Dual(self.val - other.val, self.der - other.der)
        except AttributeError:
            return Dual(self.val - other, self.der)
        
    def __rsub__(self, other):
        return Dual(other - self.val, -self.der)
        
    
    ## MULTIPLICATION OPERATIONS
    
    def __mul__(self, other):
        try:
            # multiplication rule
            temp = self.val * other.der + self.der * other.val
            return Dual(self.val * other.val, temp)
        except AttributeError:
            return Dual(self.val * other, self.der * other)

    def __rmul__(self, other):
        return Dual(self.val * other, self.der * other)
    
    
    ## DIVISION OPERATIONS
        
    def __truediv__(self, other):
        try:
            # quotient rule
            temp = (self.der * other.val - self.val * other.der)
            print(self.der)
            print(other.der)
            return Dual(self.val/other.val, temp/other.val ** 2)
        except AttributeError:
            # divide by a constant
            return Dual(self.val/other, self.der/other)
        
    def __rtruediv__(self, other):
            return Dual(other/self.val, -other/self.val**2*self.der)   
    
    
    ## POWER OPERATIONS
    
    def __pow__(self, other):
        try:
            # da^u/dx = ln(a) a^u du/dx
            factor = self.val ** (other.val -1)
            sum_1 = other.val * self.der
            sum_2 = self.val * np.log(self.val) * other.der
            temp = factor * (sum_1 + sum_2)
            return Dual(self.val ** other.val, temp)
        except AttributeError:
            # du^n/dx = n * u^(n-1) * du/dx
            temp = other * self.val ** (other-1) * self.der
            return Dual(self.val ** other, temp)
        
    def __rpow__(self, other):
            print("__rpow__")
            # da^u/dx = ln(a) a^u du/dx
            temp = np.log(other) * other ** self.val * self.der
            return Dual(other ** self.val, temp)
        


In [16]:
def exp(x):
    """Calculate the exponential of the input
        
        Keyword arguments:
        x -- a real number or a dual number
        
        Return:
        the exponential value
        """
    if (isinstance(x,Dual)):
        x.der = np.exp(x.val) * x.der
        x.val = np.exp(x.val)
        return x
    else:
        return np.exp(x)

In [17]:
trial = Dual(2)
trial_exp = exp(trial)

In [18]:
trial_exp.der #e^2

7.38905609893065

In [19]:
trial_exp.val

7.38905609893065

In [20]:
np.exp(2)

7.38905609893065

In [ ]:
from AutoDiff import AutoDiff
import admath
#from ..dual.dual import Dual

def square_fn(x):
    return x ** 2


def sin_fn(x):
    return admath.sin(5*x)+2*x**2

ad_square = AutoDiff(square_fn)
print(ad_square.get_der(3))

print(ad_square.get_der([1,2]))

ad_sin = AutoDiff(sin_fn)
print(ad_sin.get_der(2))
# Evaluates derivative of sin(5x)+2x^2 when x = 2

# test for cosine
def cos_fn(x): # trial cosine function to feed into AutoDiff
    return admath.cos(-3*x) + 3*x

ad_cos = AutoDiff(cos_fn)
print(ad_cos.get_der([2,3]))